In [1]:
from utils_ import make_toolbench_request, detect_error
import json
from error_explainer import ErrorDetector
from tqdm import tqdm
import copy

In [2]:
version = 'v4_contrastive'
data_path = f'/data4/polyakov/instruction_tuning/ToolBench/data/toolbench_new_1311/self_correction/generated_errors_no_undetectable_errors_{version}.json'
generated_data = json.load(open(data_path))
generated_data_2 = json.load(open('/data4/polyakov/instruction_tuning/ToolBench/data/toolbench_new_1311/self_correction/generated_errors_no_undetectable_errors_v_contrastive_pt2.json'))
generated_data = generated_data + generated_data_2
generated_data_3 = json.load(open('/data4/polyakov/instruction_tuning/ToolBench/data/toolbench_new_1311/self_correction/generated_errors_no_undetectable_errors_v_contrastive_pt3.json'))
generated_data = generated_data + generated_data_3
generated_data_5 = json.load(open('/data4/polyakov/instruction_tuning/ToolBench/data/toolbench_new_1311/self_correction/generated_errors_no_undetectable_errors_v_contrastive_pt5.json'))
generated_data = generated_data + generated_data_5
generated_data_6 = json.load(open('/data4/polyakov/instruction_tuning/ToolBench/data/toolbench_new_1311/self_correction/generated_errors_no_undetectable_errors_v_contrastive_pt6.json'))
generated_data = generated_data + generated_data_6
generated_data_7 = json.load(open('/data4/polyakov/instruction_tuning/ToolBench/data/toolbench_new_1311/self_correction/generated_errors_no_undetectable_errors_v_contrastive_pt7.json'))
generated_data = generated_data + generated_data_7
generated_data_8 = json.load(open('/data4/polyakov/instruction_tuning/ToolBench/data/toolbench_new_1311/self_correction/generated_errors_no_undetectable_errors_v_contrastive_pt8.json'))
generated_data = generated_data + generated_data_8

In [3]:
len(generated_data)

8989

In [4]:
llama_error_detector = ErrorDetector()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/data2/polyakov/anaconda3/envs/alpaca-lora/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/data2/polyakov/anaconda3/envs/alpaca-lora/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [7]:
generated_data_to_assess = []

for i, item in enumerate(tqdm(generated_data)):
    try:
        for step in item['predictions']:
            idx = step['chain_idx']
            gt = step['ground_truth']
            predictions = step['prediction']
            incorrect_predictions = []
            for pred in predictions:
                if pred['function_call']['name'] != 'Finish':
                    if pred['function_call']['name'] != gt['function_call']['name']:
                        incorrect_predictions.append(pred)
                    else:
                        try:
                            if json.loads(pred['function_call']['arguments']) != json.loads(gt['function_call']['arguments']):
                                incorrect_predictions.append(pred)
                        except:
                            incorrect_predictions.append(pred)
            if len(incorrect_predictions) == 2:
                if (incorrect_predictions[0]['function_call']['name'] == incorrect_predictions[1]['function_call']['name'] and 
                    incorrect_predictions[0]['function_call']['arguments'] == incorrect_predictions[1]['function_call']['arguments']):
                    incorrect_predictions = incorrect_predictions[:1]
            for pred in incorrect_predictions:
                full_prediction_text = f"\nThought: {pred['content']}\nAction: {pred['function_call']['name']}\nAction Input: {pred['function_call']['arguments']}"
                full_gt_text = f"\nThought: {gt['content']}\nAction: {gt['function_call']['name']}\nAction Input: {gt['function_call']['arguments']}"
                curr_chain = item['conversations'][:idx+2]
                gt_tool_call_result = make_toolbench_request(gt['function_call']['name'], gt['function_call']['arguments'])
                tool_call_result = make_toolbench_request(pred['function_call']['name'], pred['function_call']['arguments'])
                curr_chain.append({'from': 'assistant',
                                'value': full_prediction_text})
                curr_chain.append({'from': 'function',
                                'value': tool_call_result[0]})
                if gt_tool_call_result[1] == 0 and not detect_error(gt_tool_call_result[0]):
                    if tool_call_result[1] != 0 or detect_error(tool_call_result[0]):
                        generated_data_to_assess.append(curr_chain)
                    else:
                        if llama_error_detector.predict(full_prediction_text, tool_call_result[0]) and not llama_error_detector.predict(full_gt_text, gt_tool_call_result[0]):
                            generated_data_to_assess.append(curr_chain)
    except Exception as e:
        print(e, print(i))
        pass

 97%|███████████████████████████████████████████████████████▌ | 8761/8989 [11:19:50<17:41,  4.66s/it]


KeyboardInterrupt: 

In [21]:
correction_prompt =  "Tool call returned an error. To correct it I need to do the following. "
correction_data = []

for curr_item in generated_data_to_assess:
    item = copy.deepcopy(curr_item)
    prompt_chain = item[:-4]
    incorrect_tool_call = item[-2]
    incorrect_tool_response = item[-1]
    prompt_chain.append(incorrect_tool_call)
    prompt_chain.append(incorrect_tool_response)
    correct_tool_call = item[-4]
    correct_tool_response = item[-3]
    correct_tool_call['value'] = "\nThought: " + correction_prompt + correct_tool_call['value'].split("\nThought: ")[1]
    prompt_chain.append(correct_tool_call)
    prompt_chain.append(correct_tool_response)
    if incorrect_tool_response != correct_tool_response:
        correction_data.append(prompt_chain)
    # correction_data.append(prompt_chain)

In [22]:
len(correction_data)

2510

In [23]:
correction_data[:20]

[[{'from': 'system',
   'value': 'You are AutoGPT, you can use many tools(functions) to do the following task.\nFirst I will give you the task description, and your task start.\nAt each step, you need to give your thought to analyze the status now and what to do next, with a function call to actually excute your step. Your output should follow this format:\nThought:\nAction\nAction Input:\n\nAfter the call, you will get the call result, and you are now in a new state.\nThen you will analyze your status now, then decide what to do next...\nAfter many (Thought-call) pairs, you finally perform the task, then you can give your finial answer.\nRemember: \n1.the state change is irreversible, you can\'t go back to one of the former state, if you want to restart the task, say "I give up and restart".\n2.All the thought is short, at most in 5 sentence.\n3.You can do more then one trys, so if your plan is to continusly try some conditions, you can do one of the conditions per try.\nLet\'s Begin!

In [12]:
version = 'all_v3'
json.dump(correction_data, open(f'/data4/polyakov/instruction_tuning/ToolBench/data/toolbench_new_1311/self_correction/self_correction_data_no_undetectable_errors_{version}.json', 'w'))